In [60]:
import numpy as np
import pandas as pd
import glob
import os
os.getcwd()
os.chdir('/project/time_series')
from load_data_custom import load_data_using_multi_process
from make_roc_curve import make_roc_curve
os.chdir('/project/time_series/cchc0427')
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from model import model_lgbm
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import lazy_loader as lazy
samplerate = lazy.load("samplerate")
resampy = lazy.load("resampy")
import librosa

In [25]:
fan_path = '/data/time_series/fan'
noise_lst = glob.glob(fan_path + '/*')
id_lst = ['/id_00', '/id_02', '/id_04', '/id_06']
file_path = []
for noise in noise_lst:
    for id_num in id_lst:
        file_path.append(noise+id_num)
        

In [29]:
file_path[:4]

['/data/time_series/fan/0/id_00',
 '/data/time_series/fan/0/id_02',
 '/data/time_series/fan/0/id_04',
 '/data/time_series/fan/0/id_06']

In [9]:
sampling_lst = ['abs_max(our_custom)',
                'kaiser_best',
                'kaiser_fast',
                'fft',
                'scipy',
                'polyphase',
                'linear',
                'zero_order_hold',
                'sinc_best',
                'sinc_medium' ,
                'sinc_fastest',
                'soxr_vhq',
                'soxr_hq',
                'soxr_mq',
                'soxr_lq',
                'soxr_qq']

In [14]:
len(sampling_lst) * len(target_lst) * 12

4032

In [7]:
target_lst = []
for i in range(1, 16001):
    
    if ( i < 60 ) or ( i > 13000):
        continue
    if 16000 % i == 0 :
        target_lst.append(int(i))
        
target_lst.append(13000)
target_lst.append(16000)

In [8]:
target_lst

[64,
 80,
 100,
 125,
 128,
 160,
 200,
 250,
 320,
 400,
 500,
 640,
 800,
 1000,
 1600,
 2000,
 3200,
 4000,
 8000,
 13000,
 16000]

In [9]:
result = []
for path in file_path:
    files = glob.glob(path + '/*/*')
    obj_name = path.split('/')[-3:]
    print(f'{path} 시작')
    for sampling_way in sampling_lst:
        for target_sr in target_lst:
            fan_result = load_data_using_multi_process(files, sr = 16000, target_sr = target_sr, sampling_way = sampling_way) # 데이터 target_sr로 부르기

            fan_result_df = pd.DataFrame(fan_result)

            data = fan_result_df.iloc[:, :-1]
            target = fan_result_df.iloc[:, -1]

            X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, shuffle=True, stratify=target, random_state=42)

            model = model_lgbm(fan_result_df) # using (model.py)

            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            y_pred_prob = model.predict_proba(X_test)

            acc = accuracy_score(y_test, y_pred) 
            recall = recall_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)    
            auc_score = roc_auc_score(y_test, y_pred_prob[ :, 1 ])

            result.append([ '_'.join(obj_name), sampling_way, target_sr, round(acc, 3), round(recall, 3), round(precision, 3), round(f1, 3), round(auc_score, 3) ])
    print(f'{path} 종료')
result_df = pd.DataFrame(result, columns = ['Model', 'sampling_way', 'target_sr', 'acc', 'recall', 'precision', 'f1', 'auc_score' ])
result_df.to_csv('/project/time_series/model_all_cases.csv', index = False)            
print('파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료\n\n' * 4)

/data/time_series/fan/0/id_00 시작

멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 3.23112154006958 ---
normal : 1011
abnormal : 407
scale_pos_weight is 2.48.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 3.4008374214172363 ---
normal : 1011
abnormal : 407
scale_pos_weight is 2.48.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 3.756788969039917 ---
normal : 1011
abnormal : 407
scale_pos_weight is 2.48.
Model is ready to running.


...


멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 7.191257476806641 ---
normal : 1015
abnormal : 361
scale_pos_weight is 2.81.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



멀티 프로세싱을 시작합니다. 현재 코어 30개 사용 중. 
1.다른 사람과 동시에 돌리지 마세요.
2.함부로 진행중에 중단 하지마세요.
--- 걸린시간: 7.3514626026153564 ---
normal : 1015
abnormal : 361
scale_pos_weight is 2.81.
Model is ready to running.


/package/anaconda3.8/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


/data/time_series/fan/6/id_06 종료
파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료

파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료

파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료

파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료


